# Tutorial 03: Adding new controllers to PyCIGAR

This tutorial walks through the process of adding a new controller to PyCIGAR.

Keypoints: environment, controller.

## 1. Add a new controller checklist
To add a new controller, you need to:
- Add a controller definition with the interface similar to `pycigar/controllers/base_controller.py`.
- Run and test the new controller.

## 2. Environment

In the last tutorial, we learned how to run the simulation 1 step forward using kernel. In this tutorial, we will learn how to create environment, a standard interface to all experiment.

An environment has these methods:
- `reset()`: reset the experiment.
- `step()`: step the experiment 1 step forward (an environment step can be equivalent to `k` simulation step).

In [3]:
import numpy as np
from pycigar.core.kernel.kernel import Kernel
import yaml

class FooEnv:
    def __init__(self, sim_params, simulator='opendss'):
        """Initialize the environment.

        Parameters
        ----------
        sim_params : dict
            A dictionary of simulation information.
        simulator : str
            The name of simulator we want to use, by default it is OpenDSS.
        """
        self.state = None
        self.simulator = simulator

        # initialize the kernel
        self.k = Kernel(simulator=self.simulator,
                        sim_params=sim_params)

        # start an instance of the simulator (ex. OpenDSS)
        kernel_api = self.k.simulation.start_simulation()
        # pass the API to all sub-kernels
        self.k.pass_api(kernel_api)
        # start the corresponding scenario
        # self.k.scenario.start_scenario()



    def step(self):
        """See parent class.
        """

        for _ in range(self.sim_params['env_config']["sims_per_step"]):
            self.env_time += 1

            # perform action update for PV inverter device
            if len(self.k.device.get_pv_device_ids()) > 0:
                control_setting = []
                for device_id in self.k.device.get_pv_device_ids():
                    action = self.k.device.get_controller(device_id).get_action(self)
                    control_setting.append(action)
                self.k.device.apply_control(self.k.device.get_pv_device_ids(), control_setting)


            if self.k.time <= self.k.t:
                self.k.update(reset=False)

                # check whether the simulator sucessfully solved the powerflow
                converged = self.k.simulation.check_converged()
                if not converged:
                    break

            if self.k.time >= self.k.t:
                break

        # the episode will be finished if it is not converged.
        done = not converged or (self.k.time == self.k.t)

        return done

    def reset(self):
        self.env_time = 0
        self.k.update(reset=True)
        self.sim_params = self.k.sim_params

        self.INIT_ACTION = {}
        pv_device_ids = self.k.device.get_pv_device_ids()
        for device_id in pv_device_ids:
            self.INIT_ACTION[device_id] = np.array(self.k.device.get_control_setting(device_id))

stream = open("./data/pycigar_config.yaml", "r")
sim_params = yaml.safe_load(stream)

In [4]:
env = FooEnv(sim_params)
env.reset()
done = False
while not done:
    done = env.step()

## 3. Controller

In the `step()` methods above, we update the devices settings given by controllers. In PyCIGAR concept, one device can have 1 or more controllers. Base on the state of the environment and the device, a controller can give a *recommendation* control setting on the device, however, applying that control setting or not depends on how our choice. 

We can get the recommendation control setting from a controller by methods `get_action()` implemented in controller:
```action = self.k.device.get_controller(device_id).get_action(self)```

Apply the control settings on list of devices with:
```self.k.device.apply_control(self.k.device.get_pv_device_ids(), control_setting)```
